<a href="https://colab.research.google.com/github/weisds/resources/blob/main/Cheatsheet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Processing

##Splitting num and txt

In [ ]:
#let split the data for more targeted handling
txt_cols = [cname for cname in X.columns if df[cname].dtype == "object"]

# Select numerical columns
num_cols = [cname for cname in X.columns if df[cname].dtype in ['int64', 'float64']]

##Txt data filterin

In [ ]:
#have an overview on the txt unique data, to determine onehot, ordinal, or type transfromation 
txt_uniques = pd.DataFrame([[i, len(X[i].unique()), X[i].unique()] for i in X[txt_cols]], columns=['Variable', 'number of Unique Values', 'Unique Values']).set_index('Variable')
txt_uniques

##Ordinal

In [ ]:
#ordinal data transformation dependng on the education level
from sklearn.preprocessing import OrdinalEncoder
oen = OrdinalEncoder(categories=[['Basic','Graduation','2n Cycle', 'Master', 'PhD']])
X['Education'] = oen.fit_transform(X[['Education']])

##Nominal

In [ ]:
from sklearn.preprocessing import OneHotEncoder #OneHot Encoder
OH_en = OneHotEncoder(handle_unknown='ignore', sparse=False)
imp_txt_cols = OH_en.fit_transform(df[['Marital_Status']])
imp_txt_cols = pd.DataFrame(imp_txt_cols)
#now we align the index and col names
imp_txt_cols.index = df[['Marital_Status']].index
imp_txt_cols.columns = OH_en.get_feature_names(['Marital_Status'])
X=X.drop(['Marital_Status'], axis=1).join(imp_txt_cols)

### Splits

In [ ]:
>>> import numpy as np
>>> from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

#Modelling

##GridSearchCV

In [ ]:
parameters = {'kernel':('linear', 'rbf','poly','rbf', 'sigmoid'),
              'C': np.logspace(-3, 3, 5),
              'gamma':np.logspace(-3, 3, 5)}
svm = SVC()
svm_cv = GridSearchCV(estimator=svm, param_grid=parameters)
svm_cv.fit(X_train, y_train)

##Kmeans

In [ ]:
#elbow method to determine the best k
inertia = []
list_num_clusters = list(range(1,30))
for num_clusters in list_num_clusters:
    km = KMeans(n_clusters=num_clusters)
    km.fit(features)
    inertia.append(km.inertia_)
    
plt.plot(list_num_clusters,inertia)
plt.scatter(list_num_clusters,inertia)
plt.xlabel('Number of Clusters')
plt.ylabel('Inertia');

###KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score
f1_scores = []
for k in range(1, 50):
    # Create a KNN classifier
    knn = KNeighborsClassifier(n_neighbors=k)
    # Train the classifier
    knn = knn.fit(X_train, y_train)
    preds = knn.predict(X_test)
    # Evaluate the classifier with f1score
    f1 = f1_score(preds, y_test)
    f1_scores.append((k, round(f1_score(y_test, preds), 4)))
# Convert the f1score list to a dataframe
f1_results = pd.DataFrame(f1_scores, columns=['K', 'F1 Score'])
f1_results.set_index('K').sort_values(by = 'F1 Score', ascending=False)[:5]